In [1]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-explain-sentiment/resolve/main/explain-sentiment.json

In [2]:
from tqdm import tqdm
import json
import random
import torch
import numpy as np
from transformers import AutoTokenizer, T5Config
from malaya.torch_model.t5 import T5ForSequenceClassification

/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
with open('explain-sentiment.json') as fopen:
    data = json.load(fopen)
    
texts, labels = [], []

labels_sentiment = ['negative', 'neutral', 'positive']
for d in data:
    try:
        l = json.loads(d[1])
        if not isinstance(l['sentiment'], str):
            continue
        if l['sentiment'] not in labels_sentiment:
            continue
        if not len(d[0]):
            continue
        texts.append(d[0])
        labels.append(labels_sentiment.index(l['sentiment']))
    except:
        pass
    
len(texts)

162895

In [4]:
np.unique(labels, return_counts = True)

(array([0, 1, 2]), array([63996, 66104, 32795]))

In [5]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(
    texts, labels, test_size = 0.1
)

In [6]:
config = T5Config.from_pretrained('mesolitica/nanot5-tiny-malaysian-cased')
config.num_labels = len(set(labels))
config.vocab = labels_sentiment

In [7]:
model = T5ForSequenceClassification.from_pretrained('mesolitica/nanot5-tiny-malaysian-cased', config = config)
_ = model.cuda()

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at mesolitica/nanot5-tiny-malaysian-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-tiny-malaysian-cased')

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
trainable_parameters = [param for param in model.parameters() if param.requires_grad]
trainer = torch.optim.AdamW(trainable_parameters, lr = 2e-4)

In [12]:
batch_size = 16
epoch = 100

best_dev_acc = -np.inf
patient = 3
current_patient = 0

for e in range(epoch):
    pbar = tqdm(range(0, len(train_X), batch_size))
    losses = []
    for i in pbar:
        trainer.zero_grad()
        x = train_X[i: i + batch_size]
        y = np.array(train_Y[i: i + batch_size])
        
        padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
            
        loss, pred = model(**padded)
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(trainable_parameters, 5.0)
        trainer.step()
        losses.append(float(loss))
        
    dev_predicted = []
    for i in range(0, len(test_X), batch_size):
        x = test_X[i: i + batch_size]
        y = np.array(test_Y[i: i + batch_size])
        padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
        
        loss, pred = model(**padded)
        dev_predicted.append((pred.argmax(axis = 1).detach().cpu().numpy() == y).mean())
        
    dev_predicted = np.mean(dev_predicted)
    
    print(f'epoch: {e}, loss: {np.mean(losses)}, dev_predicted: {dev_predicted}')
    
    if dev_predicted >= best_dev_acc:
        best_dev_acc = dev_predicted
        current_patient = 0
        model.save_pretrained('tiny')
    else:
        current_patient += 1
    
    if current_patient >= patient:
        break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9163/9163 [03:38<00:00, 41.93it/s]


epoch: 0, loss: 0.5854386774347193, dev_predicted: 0.6880520117762512


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9163/9163 [03:35<00:00, 42.45it/s]


epoch: 1, loss: 0.5755012557130056, dev_predicted: 0.6909960745829244


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9163/9163 [03:36<00:00, 42.38it/s]


epoch: 2, loss: 0.5675006818299104, dev_predicted: 0.6892173699705594


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9163/9163 [03:39<00:00, 41.79it/s]


epoch: 3, loss: 0.5606617437575785, dev_predicted: 0.6868253189401374


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 9163/9163 [03:35<00:00, 42.44it/s]


epoch: 4, loss: 0.5501954948425319, dev_predicted: 0.6884813542688911


In [13]:
model_ = T5ForSequenceClassification.from_pretrained('tiny')
_ = model_.cuda()

In [14]:
real_Y = []
for i in tqdm(range(0, len(test_X), batch_size)):
    x = test_X[i: i + batch_size]
    y = np.array(test_Y[i: i + batch_size])
    padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
    padded['labels'] = torch.from_numpy(y)
    for k in padded.keys():
        padded[k] = padded[k].cuda()

    loss, pred = model(**padded)
    real_Y.extend(pred.argmax(axis = 1).detach().cpu().numpy().tolist())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1019/1019 [00:05<00:00, 169.95it/s]


In [15]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, test_Y, target_names = labels_sentiment,
        digits = 5
    )
)

              precision    recall  f1-score   support

    negative    0.74369   0.72648   0.73499      6570
     neutral    0.67019   0.66743   0.66881      6546
    positive    0.61915   0.65406   0.63613      3174

    accuracy                        0.68864     16290
   macro avg    0.67768   0.68266   0.67997     16290
weighted avg    0.68989   0.68864   0.68913     16290



In [16]:
model_.push_to_hub('mesolitica/sentiment-analysis-nanot5-tiny-malaysian-cased', safe_serialization = True)

model.safetensors:   0%|          | 0.00/93.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/sentiment-analysis-nanot5-tiny-malaysian-cased/commit/73de716fd900bf7f467fb3dad467954e857f5635', commit_message='Upload T5ForSequenceClassification', commit_description='', oid='73de716fd900bf7f467fb3dad467954e857f5635', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
tokenizer.push_to_hub('mesolitica/sentiment-analysis-nanot5-tiny-malaysian-cased', safe_serialization = True)

CommitInfo(commit_url='https://huggingface.co/mesolitica/sentiment-analysis-nanot5-tiny-malaysian-cased/commit/32f04a30b42c8c84957fdc28bb9aa04bb411fc35', commit_message='Upload tokenizer', commit_description='', oid='32f04a30b42c8c84957fdc28bb9aa04bb411fc35', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
labels_sentiment

['negative', 'neutral', 'positive']